# 1. Build docker image

In [3]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [50]:
!docker build -t grammer-classification-bert-base-uncased -f Dockerfile .

Sending build context to Docker daemon  2.564MB
Step 1/12 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.0-cpu-py36-ubuntu16.04
 ---> e77085e0f844
Step 2/12 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> c6e213b7a95d
Step 3/12 : COPY ./code /opt/ml/code
 ---> eee743023e35
Step 4/12 : RUN mkdir -p /opt/ml/model &&     ls -l /opt/ml/code &&     pip install -r /opt/ml/code/requirements.txt
 ---> Running in 7dad89e3f0e9
total 32
drwxrwxr-x 2 root root  4096 Nov 29 14:41 __pycache__
-rw-rw-r-- 1 root root  3226 Nov 29 14:41 deploy_ei.py
-rw-rw-r-- 1 root root    77 Dec  2 06:44 requirements.txt
-rw-rw-r-- 1 root root  1320 Nov 29 14:41 test.py
-rw-rw-r-- 1 root root 13160 Dec  2 07:47 train_deploy.py
  Created wheel for aiobotocore: filename=aiobotocore-2.0.1-py3-none-any.whl size=51978 sha256=4c2f739ed62a3c909d4db9ee780f87614168ccac2ecfa7bd78280396e35fba8a
  Stored in directory: /root/.cache/pip/wheels/73/b7/3b/15481dc56d092eba140bebca5486d592b34b90f93b

In [ ]:
!docker run --name ivy-grammer-classification grammer-classification-bert-base-uncased:latest

# 2. Push to ECR

In [53]:
import boto3

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = "grammer-classification-bert-base-uncased"

ecr_image = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name)

print(ecr_image)

420737321821.dkr.ecr.us-east-1.amazonaws.com/grammer-classification-bert-base-uncased:latest


In [ ]:
# !aws ecr get-login-password | docker login xxxx -U AWS --password-stdin
# !docker tag pytorch-bert-base-uncased:latest ECR_IMAGE 
# !docker push ECR_IMAGE

In [54]:
%%sh

# The name of our algorithm
algorithm_name=grammer-classification-bert-base-uncased

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# # Get the login command from ECR in order to pull down the SageMaker PyTorch image
# $(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)

# # Build the docker image locally with the image name and then push it to ECR
# # with the full name.

aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com

# docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Login Succeeded
The push refers to repository [420737321821.dkr.ecr.us-east-1.amazonaws.com/grammer-classification-bert-base-uncased]
0bf23823ffca: Preparing
0b6d4fed022e: Preparing
6160e55345f6: Preparing
b8897f99fee4: Preparing
dc1b4f416d27: Preparing
912a9194a89a: Preparing
0bb6ba7c32f9: Preparing
d6b6f976152c: Preparing
bd9ea4625f76: Preparing
31aed7f45c93: Preparing
2dc64f428e8f: Preparing
6225f83539e8: Preparing
dbcf411e8bf0: Preparing
5276d2b930fc: Preparing
e6feec0db89a: Preparing
697949baa658: Preparing
935c56d8b3f9: Preparing
31aed7f45c93: Waiting
2dc64f428e8f: Waiting
6225f83539e8: Waiting
dbcf411e8bf0: Waiting
5276d2b930fc: Waiting
e6feec0db89a: Waiting
697949baa658: Waiting
935c56d8b3f9: Waiting
912a9194a89a: Waiting
0bb6ba7c32f9: Waiting
d6b6f976152c: Waiting
bd9ea4625f76: Waiting


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

denied: User: arn:aws:sts::420737321821:assumed-role/CUSPFE-SageMaker-ML-Team-Execution-Role-Test/SageMaker is not authorized to perform: ecr:InitiateLayerUpload on resource: arn:aws:ecr:us-east-1:420737321821:repository/grammer-classification-bert-base-uncased because no identity-based policy allows the ecr:InitiateLayerUpload action


CalledProcessError: Command 'b'\n# The name of our algorithm\nalgorithm_name=grammer-classification-bert-base-uncased\n\naccount=$(aws sts get-caller-identity --query Account --output text)\n\n# Get the region defined in the current configuration (default to us-west-2 if none defined)\nregion=$(aws configure get region)\nregion=${region:-us-west-2}\n\nfullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"\n\n# If the repository doesn\'t exist in ECR, create it.\n\naws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1\n\nif [ $? -ne 0 ]\nthen\n    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null\nfi\n\n# Get the login command from ECR and execute it directly\n$(aws ecr get-login --region ${region} --no-include-email)\n\n# # Get the login command from ECR in order to pull down the SageMaker PyTorch image\n# $(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)\n\n# # Build 

# 3. Deploy to Sagemaker Endpoint

In [48]:
import os
import numpy as np
import pandas as pd
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/ivy-demo-pytorch-bert"

role = sagemaker.get_execution_role()

model_data="s3://{}/{}/pytorch-training-2021-11-29-14-48-31-535/output/model.tar.gz".format(bucket, prefix)

print(bucket)
print(role)
print(model_data)

sagemaker-us-east-1-420737321821
arn:aws:iam::420737321821:role/CUSPFE-SageMaker-ML-Team-Execution-Role-Test
s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/pytorch-training-2021-11-29-14-48-31-535/output/model.tar.gz


In [29]:
# help(sagemaker_session.upload_data)

In [14]:
!aws s3 ls s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/pytorch-training-2021-11-29-14-48-31-535/output/model.tar.gz

2021-11-29 14:55:18  405589243 model.tar.gz


## 3.1 Creating a model from training output (model artifacts)

In [15]:
model_name_prefix = "{}-model".format(algorithm_name)
timestamp = time.strftime('-%Y%m%d%H%M%S', time.gmtime())
model_name = model_name_prefix + timestamp
print(model_name)

grammer-classification-bert-base-uncased-model-20211202031053


In [16]:
sagemaker_client = boto3.client(service_name='sagemaker')

model_artifacts_s3_path = model_data
docker_image = algorithm_name

create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': docker_image,
        'ModelDataUrl': model_artifacts_s3_path,
    }
)

print(create_model_response['ModelArn'])

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Using non-ECR image "grammer-classification-bert-base-uncased" without Vpc repository access mode is not supported.

In [14]:
len("grammer-classification-bert-base-uncased-model-2021-12-02-03-09-55")

66

## 3.1. Create Pytorch endpoint with predefined the pytorch framework
- it will create the inference model and endpoint configure automatically
- it will use the awe prebuilt pytorch framework, we don't need to build our own docker imag, but need to prepare the entrypoint script

In [39]:
# from sagemaker.pytorch import PyTorchModel
# help(PyTorchModel)

import sagemaker
# help(sagemaker.model.FrameworkModel)
# help(sagemaker.model.Model)

# from sagemaker.estimator import Estimator
# help(Estimator)

# help(estimator.fit)

In [31]:
WORK_DIRECTORY = "/home/ec2-user/SageMaker/amazon-sagemaker-bert-pytorch/cola_public"
my_prefix="sagemaker/ivy-demo-pytorch-bert/data/input"
data_location = sagemaker_session.upload_data(WORK_DIRECTORY, bucket=bucket, key_prefix=my_prefix)
print(data_location)

s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert


In [34]:
!aws s3 mv s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/test.csv s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/data/input/test.csv 
!aws s3 mv s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/train.csv s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/data/input/train.csv 
!aws s3 ls s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/

move: s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/test.csv to s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/data/input/test.csv
move: s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/train.csv to s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/data/input/train.csv
                           PRE data/
                           PRE pytorch-training-2021-11-29-14-48-31-535/
                           PRE raw/
                           PRE tokenized/
2021-12-02 06:32:45       6570 README


In [52]:
from sagemaker.estimator import Estimator
import time

instance_type = 'ml.m5.large'
# accelerator_type = 'ml.eia2.xlarge'
# instance_type = 'ml.t2.medium'
hyperparameters = {"epochs": 1}

estimator = Estimator(
    role=role,
    train_instance_count=1,
    train_instance_type=instance_type,
    image_uri="grammer-classification-bert-base-uncased:latest",
    model_uri=model_data,
    hyperparameters=hyperparameters,
)

estimator.fit()
# estimator.fit("s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/data/input/")


# estimator.fit("file:///tmp/pytorch-example/cifar-10-data")
endpoint_name = 'grammar-classification-{}-ep'.format(time.time()).replace('.', '').replace('_', '')
print(endpoint_name)
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="local", #instance_type,
#     accelerator_type=accelerator_type,
    endpoint_name=endpoint_name,
    wait=True,
)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: TrainingImageConfig with TrainingRepositoryAccessMode set to VPC must be provided when using a training image from a private Docker registry. Please provideTrainingImageConfig and TrainingRepositoryAccessMode set to VPC when using a training image from a private Docker registry.

In [5]:
from sagemaker.pytorch import PyTorchModel
import time

# instance_type = 'ml.m5.large'
# accelerator_type = 'ml.eia2.xlarge'
instance_type = 'ml.t2.medium'

endpoint_name = 'grammar-classification-{}-ep'.format(time.time()).replace('.', '').replace('_', '')
print(endpoint_name)
pytorch = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point='train_deploy.py',
    source_dir='code',
#     framework_version='1.3.1',
#     py_version='py3',
    image_uri="grammer-classification-bert-base-uncased:latest",
    sagemaker_session=sagemaker_session
)

# Function will exit before endpoint is finished creating
predictor = pytorch.deploy(
    initial_instance_count=1,
    instance_type="local", #instance_type,
#     accelerator_type=accelerator_type,
    endpoint_name=endpoint_name,
    wait=True,
)

grammar-classification-1638413436289433-ep


TypeError: expected string or bytes-like object

In [6]:
# from sagemaker.estimator import Estimator
# help(Estimator.fit)

In [ ]:
from sagemaker.estimator import Estimator

# hyperparameters = {"epochs": 1}
hyperparameters={
        "epochs": 1,
        "num_labels": 2,
        "backend": "gloo",
        "lr":0.1
    }

instance_type = "ml.m4.xlarge"
model_uri="s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/pytorch-training-2021-11-29-14-48-31-535/output/model.tar.gz"
estimator = Estimator(
    role=role,
    train_instance_count=1,
    train_instance_type=instance_type,
    image_name=ecr_image,
    model_uri=model_uri,
    hyperparameters=hyperparameters,
)

estimator.fit()

predictor = estimator.deploy(1, instance_type)

In [9]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(boto3.Session().region_name, 'image-classification')
print(training_image)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


# 4. Test the endpoint